In [1]:
%%html
<style>
table {float:left}
</style>

In [3]:
# my python files
import src.analytics as tics
import src.plotting_functions as pltfuncs
import src.transforms as trans
import src.hyp_test as hyp

# Imports
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import scipy.stats as stats
from scipy.stats import kendalltau
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [5]:
# On OSX requires brew install coreutils, using gshuf instead of shuf
sample_name = 'data/sample_random_subset.csv'
tics.create_random_samples('data/kddcup.data', 10000, sample_name)

df = trans.read_data(sample_name)
df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,icmp,ecr_i,SF,1032,0,0,0,0,0,...,255,1.00,0.00,1.00,0.00,0.00,0.00,0.0,0.0,smurf.
1,0,icmp,ecr_i,SF,1032,0,0,0,0,0,...,255,1.00,0.00,1.00,0.00,0.00,0.00,0.0,0.0,smurf.
2,0,tcp,private,S0,0,0,0,0,0,0,...,9,0.04,0.05,0.00,0.00,1.00,1.00,0.0,0.0,neptune.
3,0,tcp,private,S0,0,0,0,0,0,0,...,17,0.07,0.05,0.00,0.00,1.00,1.00,0.0,0.0,neptune.
4,2,tcp,smtp,SF,958,283,0,0,0,0,...,177,0.83,0.06,0.01,0.01,0.02,0.01,0.0,0.0,normal.


### Dropping Features


##### Highly Correlated Features
```
dst_host_srv_serror_rate     srv_serror_rate                0.999370
dst_host_srv_serror_rate     dst_host_serror_rate           0.999165
srv_serror_rate              serror_rate                    0.999086
dst_host_serror_rate         serror_rate                    0.999039
serror_rate                  dst_host_serror_rate           0.999039
dst_host_srv_serror_rate     serror_rate                    0.998765
dst_host_serror_rate         srv_serror_rate                0.998761
srv_rerror_rate              rerror_rate                    0.997359
srv_rerror_rate              dst_host_srv_rerror_rate       0.989355
rerror_rate                  dst_host_srv_rerror_rate       0.988520
rerror_rate                  dst_host_rerror_rate           0.986976
dst_host_rerror_rate         dst_host_srv_rerror_rate       0.985911
srv_rerror_rate              dst_host_rerror_rate           0.984211
dst_host_srv_count           dst_host_same_srv_rate         0.976918
dst_host_same_src_port_rate  srv_count                      0.947230
count                        srv_count                      0.944881
dst_host_same_srv_rate       same_srv_rate                  0.929902
same_srv_rate                dst_host_srv_count             0.904395
same_srv_rate                dst_host_serror_rate           0.866402
serror_rate                  same_srv_rate                  0.866047
count                        dst_host_same_src_port_rate    0.865827
dst_host_srv_serror_rate     same_srv_rate                  0.865370
srv_serror_rate              same_srv_rate                  0.864885
dst_host_same_srv_rate       dst_host_serror_rate           0.808542
dst_host_same_srv_rate       serror_rate                    0.807922
dst_host_srv_serror_rate     dst_host_same_srv_rate         0.807354
dst_host_same_srv_rate       srv_serror_rate                0.806843
```


In [8]:

counts = tics.count_categories(df)

In [9]:
counts

{'count': 341,
 'diff_srv_rate': 36,
 'dst_bytes': 1048,
 'dst_host_count': 233,
 'dst_host_diff_srv_rate': 73,
 'dst_host_rerror_rate': 49,
 'dst_host_same_src_port_rate': 81,
 'dst_host_same_srv_rate': 100,
 'dst_host_serror_rate': 24,
 'dst_host_srv_count': 229,
 'dst_host_srv_diff_host_rate': 38,
 'dst_host_srv_rerror_rate': 42,
 'dst_host_srv_serror_rate': 12,
 'duration': 114,
 'flag': 8,
 'hot': 6,
 'is_guest_login': 2,
 'is_host_login': 1,
 'label': 11,
 'land': 1,
 'logged_in': 2,
 'num_access_files': 2,
 'num_compromised': 2,
 'num_failed_logins': 2,
 'num_file_creations': 3,
 'num_outbound_cmds': 1,
 'num_root': 5,
 'num_shells': 2,
 'protocol_type': 3,
 'rerror_rate': 25,
 'root_shell': 1,
 'same_srv_rate': 51,
 'serror_rate': 22,
 'service': 54,
 'src_bytes': 518,
 'srv_count': 173,
 'srv_diff_host_rate': 44,
 'srv_rerror_rate': 15,
 'srv_serror_rate': 16,
 'su_attempted': 1,
 'urgent': 1,
 'wrong_fragment': 3}